In [18]:
# solution to puzzles described here http://adventofcode.com/2017/day/7

# return (name, weight) from pattern `name (weight)`
def parse_head(line):
    result = line.translate(None, '()').split(' ')
    return (result[0], int(result[1]))

# function that parses data
def parse_program(line):
    if line.find('->') == -1:
        name, weight = parse_head(line)
        return name, (weight, [])
    else:
        # split head from tail list
        split_line = line.strip().split(' -> ')
        name, weight = parse_head(split_line[0])
        tail = split_line[1].split(', ')
        return name, (weight, tail)

# read data
programs_raw = open('day7_input.txt').readlines()
programs = {key: value for key,value in map(parse_program, programs_raw)}

In [51]:
### PUZZLE 1 ###

# get programs that support some other program
base_programs = {key for key, value in programs.items() if value[1] != []}

# get set of programs linked by somemone else
linked_programs = set()
for key in base_programs:
    linked_programs.update(programs[key][1])
    
# the set difference should be the answer
result = base_programs.difference(linked_programs)
assert(len(result) == 1)

print 'The answer to the first puzzle is', result.pop()

The answer to the first puzzle is  fbgguv


In [103]:
### PUZZLE 2 ###

# get weight of program with all supported programs
def total_weight(prog, programs, cache):
    # check memoized results
    if prog in cache:
        return cache[prog]
    
    data = programs[prog]
    own_weight = data[0]
    supp = data[1]
    
    # return own weight if there are no supported programs
    if supp == []:
        cache[prog] = own_weight
        return own_weight
    
    result = own_weight + sum([total_weight(x, programs, cache) for x in supp])
    cache[prog] = result
    return result

cache = {}  # start empty cache

def fix_wrong(named_weights):
    
    # make histogram of weights
    dd = {}
    for name, weight in named_weights:
        if weight in dd:
            dd[weight] += name
        else:
            dd[weight] = [name]
    
    # if there is only one entry, return -1 (no error)
    if len(dd) == 1:
        return -1
    
    # othwerwise return entry with frequency > 1
    correct_weight = [key for key, value in dd.items() if len(value) > 1][0]
    wrong_weight, name_wrong = [(key, value) for key, value in dd.items() if len(value) == 1][0]
    return (correct_weight - wrong_weight, wrong_weight, name_wrong[0])


# loop over all programs that support others to find inconsistencies
# put them all in a list of (correct weight, current total weight, name)
errors = []
for prog in base_programs:
    data = programs[prog]
    supp_weights = [(x, total_weight(x, programs, cache)) for x in data[1]]
    
    correction = fix_wrong(supp_weights)
    if correction != -1:
        program_at_fault = correction[2]        ## name
        total = correction[1]                   ## total weight (to find deepest)
        current_weight_at_fault = programs[program_at_fault][0]  ## own weight
        result = current_weight_at_fault + correction[0]  ## assuming this is deepest
        errors.append((result, total, program_at_fault))  ## put in list to sort later
        
# order errors so the deepest one comes to the front
errors.sort(key=lambda x: x[1])  ## second value is total weight

print errors
print 'The answer to the second puzzle is', errors[0][0]

[(1864, 2553, 'jdxfsa'), (72, 17918, 'zsasjr'), (57, 89632, 'sfruur')]
The answer to the second puzzle is 1864
